In [36]:
#import dependencies
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress

##### Time-based Features:

- Extract day of the week, month, and year from the Date column.
- Create binary features indicating whether the day is a weekend or a weekday.
- Create a feature representing the week of the year.

##### Promotion Features:

- Create a binary feature indicating whether Promo and Promo2 are both active.
- Create a binary feature indicating whether promo2 is active on a given day, based on the PromoInterval and the current month.

##### Competition Features:

- Calculate the number of months since the competition store opened.

##### Handling categirocal variables:

- Encoding Categorical Features: One-hot encode categorical features such as StoreType, Assortment and StateHoliday.


In [37]:
#Read in data
merged_df = pd.read_csv("Resources/cleaned_merged_df.csv")

C:\Users\Beaut\AppData\Local\Temp\ipykernel_100\1426173617.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.read_csv("Resources/cleaned_merged_df.csv")


In [38]:
#Time-based Feature
merged_df['Date'] = pd.to_datetime(merged_df['Date'])

# Extract day of the week, month, year, and week of the year from the 'Date' column
merged_df['DayOfWeek'] = merged_df['Date'].dt.dayofweek + 1  # +1 to make Monday=1, Sunday=7
merged_df['Month'] = merged_df['Date'].dt.month
merged_df['Year'] = merged_df['Date'].dt.year
merged_df['WeekOfYear'] = merged_df['Date'].dt.isocalendar().week

# Create binary features indicating whether the day is a weekend or a weekday
merged_df['IsWeekend'] = merged_df['DayOfWeek'].isin([6, 7]).astype(int)  # 6=Saturday, 7=Sunday
merged_df['IsWeekday'] = (~merged_df['DayOfWeek'].isin([6, 7])).astype(int)  # Not Saturday or Sunday

# Display the first few rows of the updated DataFrame
merged_df.head()



,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,Promo2SinceWeek,Promo2SinceYear,PromoInterval,CompetitionOpenSinceMonth_Missing,CompetitionOpenSinceYear_Missing,Month,Year,WeekOfYear,IsWeekend,IsWeekday
0,1,5,2015-07-31,5263,555,1,1,0,1,c,...,0.0,0.0,No Promo2,0,0,7,2015,31,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1,a,...,13.0,2010.0,"Jan,Apr,Jul,Oct",0,0,7,2015,31,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1,a,...,14.0,2011.0,"Jan,Apr,Jul,Oct",0,0,7,2015,31,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,...,0.0,0.0,No Promo2,0,0,7,2015,31,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1,a,...,0.0,0.0,No Promo2,0,0,7,2015,31,0,1


In [39]:
# promotional Feature

# Create a binary feature indicating whether 'Promo' and 'Promo2' are both active
merged_df['PromoAndPromo2Active'] = ((merged_df['Promo'] == 1) & (merged_df['Promo2'] == 1)).astype(int)

# Display the first few rows of the updated DataFrame
merged_df.head()


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,Promo2SinceYear,PromoInterval,CompetitionOpenSinceMonth_Missing,CompetitionOpenSinceYear_Missing,Month,Year,WeekOfYear,IsWeekend,IsWeekday,PromoAndPromo2Active
0,1,5,2015-07-31,5263,555,1,1,0,1,c,...,0.0,No Promo2,0,0,7,2015,31,0,1,0
1,2,5,2015-07-31,6064,625,1,1,0,1,a,...,2010.0,"Jan,Apr,Jul,Oct",0,0,7,2015,31,0,1,1
2,3,5,2015-07-31,8314,821,1,1,0,1,a,...,2011.0,"Jan,Apr,Jul,Oct",0,0,7,2015,31,0,1,1
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,...,0.0,No Promo2,0,0,7,2015,31,0,1,0
4,5,5,2015-07-31,4822,559,1,1,0,1,a,...,0.0,No Promo2,0,0,7,2015,31,0,1,0


In [40]:
#promotional features 

# Define the month_mapping dictionary
month_mapping = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}

# Handle missing values in 'PromoInterval' and replace 'Sept' with 'Sep'
merged_df['PromoInterval'] = merged_df['PromoInterval'].fillna('No Promo2').str.replace('Sept', 'Sep')

# Convert the 'PromoInterval' column to contain the names of the starting months of Promo2
merged_df['PromoInterval'] = merged_df['PromoInterval'].apply(lambda x: [month_mapping[month] for month in x.split(',')] if x != 'No Promo2' else [])

# Define a function to determine whether Promo2 is active on a given date based on the PromoInterval
def is_promo2_active(row):
    if row['Promo2'] == 0:
        return 0
    elif row['Month'] in row['PromoInterval']:
        return 1
    else:
        return 0

# Create a binary feature 'IsPromo2Active' indicating whether Promo2 is active on a given day
merged_df['IsPromo2Active'] = merged_df.apply(is_promo2_active, axis=1)

# Display the first few rows of the updated DataFrame
merged_df.head()


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,PromoInterval,CompetitionOpenSinceMonth_Missing,CompetitionOpenSinceYear_Missing,Month,Year,WeekOfYear,IsWeekend,IsWeekday,PromoAndPromo2Active,IsPromo2Active
0,1,5,2015-07-31,5263,555,1,1,0,1,c,...,[],0,0,7,2015,31,0,1,0,0
1,2,5,2015-07-31,6064,625,1,1,0,1,a,...,"[1, 4, 7, 10]",0,0,7,2015,31,0,1,1,1
2,3,5,2015-07-31,8314,821,1,1,0,1,a,...,"[1, 4, 7, 10]",0,0,7,2015,31,0,1,1,1
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,...,[],0,0,7,2015,31,0,1,0,0
4,5,5,2015-07-31,4822,559,1,1,0,1,a,...,[],0,0,7,2015,31,0,1,0,0


In [41]:
#Competition Feature

# Convert 'CompetitionOpenSinceYear' and 'CompetitionOpenSinceMonth' to string
# Fill NaN values in 'CompetitionOpenSinceYear' and 'CompetitionOpenSinceMonth'
merged_df['CompetitionOpenSinceYear'].fillna(merged_df['Year'], inplace=True)
merged_df['CompetitionOpenSinceMonth'].fillna(1, inplace=True)

# Convert 'CompetitionOpenSinceYear' and 'CompetitionOpenSinceMonth' to string
merged_df['CompetitionOpenSinceYear'] = merged_df['CompetitionOpenSinceYear'].astype(int).astype(str)
merged_df['CompetitionOpenSinceMonth'] = merged_df['CompetitionOpenSinceMonth'].astype(int).astype(str)

# Create 'competition_open_month_year' by joining year and month strings and convert to 'Month' period
competition_open_month_year = (merged_df['CompetitionOpenSinceYear'] + '-' + 
                               merged_df['CompetitionOpenSinceMonth'])
competition_open_month_year = pd.to_datetime(competition_open_month_year, errors='coerce').dt.to_period('M')

# Calculate 'MonthsSinceCompetitionOpen' and fill NA with 0
current_month_year = merged_df['Date'].dt.to_period('M')
months_since_competition_open = (current_month_year - competition_open_month_year).apply(lambda x: x.n if pd.notna(x) else 0)
merged_df['MonthsSinceCompetitionOpen'] = months_since_competition_open

# Display the first few rows of the updated DataFrame
merged_df.head()


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,CompetitionOpenSinceMonth_Missing,CompetitionOpenSinceYear_Missing,Month,Year,WeekOfYear,IsWeekend,IsWeekday,PromoAndPromo2Active,IsPromo2Active,MonthsSinceCompetitionOpen
0,1,5,2015-07-31,5263,555,1,1,0,1,c,...,0,0,7,2015,31,0,1,0,0,82
1,2,5,2015-07-31,6064,625,1,1,0,1,a,...,0,0,7,2015,31,0,1,1,1,92
2,3,5,2015-07-31,8314,821,1,1,0,1,a,...,0,0,7,2015,31,0,1,1,1,103
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,...,0,0,7,2015,31,0,1,0,0,70
4,5,5,2015-07-31,4822,559,1,1,0,1,a,...,0,0,7,2015,31,0,1,0,0,3


In [42]:
#One-hot Encoding Categorical Features

# Convert all representations of "no state holiday" to a common format, e.g., the string "0"
merged_df['StateHoliday'] = merged_df['StateHoliday'].apply(lambda x: '0' if x == 0 else x)

# One-hot encode the 'StateHoliday', 'StoreType', and 'Assortment' columns
encoded_columns = pd.get_dummies(merged_df[['StoreType', 'Assortment']], drop_first=False)

# Perform one-hot encoding again
state_holiday_dummies = pd.get_dummies(merged_df['StateHoliday'], prefix='StateHoliday')

# Concatenate the original DataFrame with the encoded columns
merged_df = pd.concat([merged_df, encoded_columns], axis=1)

merged_df = pd.concat([merged_df, state_holiday_dummies], axis=1)


# Drop the original categorical columns
merged_df = merged_df.drop(columns=['StateHoliday', 'StoreType', 'Assortment'])



In [43]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 36 columns):
 #   Column                             Non-Null Count    Dtype         
---  ------                             --------------    -----         
 0   Store                              1017209 non-null  int64         
 1   DayOfWeek                          1017209 non-null  int64         
 2   Date                               1017209 non-null  datetime64[ns]
 3   Sales                              1017209 non-null  int64         
 4   Customers                          1017209 non-null  int64         
 5   Open                               1017209 non-null  int64         
 6   Promo                              1017209 non-null  int64         
 7   SchoolHoliday                      1017209 non-null  int64         
 8   CompetitionDistance                1017209 non-null  float64       
 9   CompetitionOpenSinceMonth          1017209 non-null  object        
 10  Compet

In [44]:
print(len(merged_df.columns))

36


In [46]:
merged_df.to_csv("Resources/feature_engineered_data.csv", index = False)